In [1]:
'''
    Data isnt evently distributed accross the executors

    Salting: add a random prefix/suffix to key in order to spread records accross multiple partitions
'''

'\n    Data isnt evently distributed accross the executors\n\n    Salting: add a random prefix/suffix to key in order to spread records accross multiple partitions\n'

In [2]:
from common.session import get_spark_session

spark = get_spark_session('Data skewed')

In [15]:
sales_data = [('A',i) for i in range(0,100)] + \
    [('B',i) for i in range(1,5)] + \
    [('C',i) for i in range(1,5)]  \


sales_cols = ["user_id","amount"]

df_sales = spark.createDataFrame(sales_data, sales_cols)

from pyspark.sql.functions import count
df_sales.groupBy("user_id").agg(count("user_id").alias("total_records")).show()

+-------+-------------+
|user_id|total_records|
+-------+-------------+
|      A|          100|
|      B|            4|
|      C|            4|
+-------+-------------+



In [16]:
# salting
from pyspark.sql.functions import expr, concat_ws

df_sales_salted = df.withColumn("salt", expr("floor(rand() * 10)")).withColumn("salted_key", concat_ws("_", "user_id", "salt"))
df_sales_salted.show()



+-------+------+----+----------+
|user_id|amount|salt|salted_key|
+-------+------+----+----------+
|      A|     1|   1|       A_1|
|      A|     2|   8|       A_8|
|      A|     3|   1|       A_1|
|      A|     4|   5|       A_5|
|      A|     5|   3|       A_3|
|      A|     6|   8|       A_8|
|      A|     7|   3|       A_3|
|      A|     8|   4|       A_4|
|      A|     9|   8|       A_8|
|      A|    10|   3|       A_3|
|      A|    11|   5|       A_5|
|      A|    12|   3|       A_3|
|      A|    13|   1|       A_1|
|      A|    14|   9|       A_9|
|      A|    15|   4|       A_4|
|      A|    16|   7|       A_7|
|      A|    17|   4|       A_4|
|      A|    18|   3|       A_3|
|      A|    19|   9|       A_9|
|      A|    20|   2|       A_2|
+-------+------+----+----------+
only showing top 20 rows



In [17]:
df_salted.groupBy("salted_key").agg(count("user_id").alias("total_records")).show()

+----------+-------------+
|salted_key|total_records|
+----------+-------------+
|       A_4|           11|
|       A_7|            6|
|       A_0|            9|
|       A_6|            9|
|       A_3|           12|
|       A_1|           16|
|       A_5|           10|
|       A_9|            9|
|       A_2|            9|
|       A_8|            8|
|       C_1|            1|
|       B_6|            1|
|       B_9|            1|
|       C_3|            1|
|       B_2|            1|
|       B_0|            1|
|       C_6|            1|
|       C_7|            1|
+----------+-------------+

